<a href="https://colab.research.google.com/github/forest1102/Keras-Test/blob/master/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Requirements:

In [0]:
!pip install tensorflow==2.0.0b1 numpy

     |████████████████████████████████| 87.9MB 51kB/s 
     |████████████████████████████████| 3.1MB 34.1MB/s 
     |████████████████████████████████| 501kB 59.7MB/s 
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!mkdir dataset
!wget https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt -P /content/dataset
!wget https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.test.txt -P /content/dataset

mkdir: cannot create directory ‘dataset’: File exists
--2019-12-30 12:39:03--  https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5101618 (4.9M) [text/plain]
Saving to: ‘/content/dataset/ptb.train.txt.1’

ptb.train.txt.1     100%[===================>]   4.87M  --.-KB/s    in 0.08s   

2019-12-30 12:39:04 (62.0 MB/s) - ‘/content/dataset/ptb.train.txt.1’ saved [5101618/5101618]

--2019-12-30 12:39:06--  https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, Lambda, Input, Reshape, Dot, Flatten
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard

import pickle

from util import create_contexts_target, most_similar
from dataset import ptb
from negative_sampling import generate_with_negative_sample

tensorboard_callback = TensorBoard(log_dir='logs/cbow')

window_size = 10
hidden_size = 100
batch_size = 100
max_epoch = 15
sample_size = 5

corpus, word_to_id, id_to_word = ptb.load_data('train')
test_corpus = ptb.load_data('test')[0]

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
test_contexts, test_target = create_contexts_target(test_corpus, window_size)

print('corpus', corpus.shape)
print('contexts', contexts.shape)
print('target', target.shape)


contexts_input = Input(shape=(window_size * 2,), name='contexts_input')
target_input = Input(shape=(1,), name='target_input')

embed = Embedding(vocab_size, hidden_size, input_length=window_size * 2)

contexts_embed = embed(contexts_input)
contexts_hidden = Lambda(lambda arr: K.mean(arr, axis=1))(contexts_embed)

target_embed = Embedding(vocab_size, hidden_size, input_length=1)(target_input)
target_hidden = Reshape((hidden_size, ))(target_embed)

embed_dot = Dot(axes=1)([contexts_hidden, target_hidden])
output = Dense(1, activation='sigmoid')(embed_dot)

model = Model(inputs=[contexts_input, target_input], outputs=output)
print('corpus', corpus.shape)
print('contexts', contexts.shape)
print('target', target.shape)

model.compile(
    optimizer='Adam',
    loss='binary_crossentropy',
    metrics=['acc']
)

print(model.summary())
hist = model.fit_generator(
    generate_with_negative_sample(
        corpus, contexts, target, batch_size, sample_size=sample_size),
    steps_per_epoch=len(contexts) // batch_size,
    initial_epoch=0,
    epochs=max_epoch, callbacks=[tensorboard_callback],
)

word_vecs = model.get_weights()[0]
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)



/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

corpus (929589,)
contexts (929569, 20)
target (929569, 1)
corpus (929589,)
contexts (929569, 20)
target (929569, 1)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
contexts_input (InputLayer)     [(None, 20)]         0                                            
__________________________________________________________________________________________________
target_input (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 100)      1000000     contexts_input[0][0]             
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 100)       1000000     target_input